In [19]:
import numpy as np

In [25]:
v = np.array([-4,-3,8])
b11 = np.array([1,2,3])
b22 = np.array([-2,1,0])
b33 = np.array([-3,-6,5])

b1 = b11 / np.linalg.norm(b11)
b2 = b22 / np.linalg.norm(b22)
b3 = b33 / np.linalg.norm(b33)

proj_b1 = np.linalg.norm((np.dot(v,b1)* b1) / (np.linalg.norm(b1)**2))
proj_b2 = np.linalg.norm((np.dot(v,b2)* b2) / (np.linalg.norm(b2)**2))
proj_b3 = np.linalg.norm((np.dot(v,b3)* b3) / (np.linalg.norm(b3)**2))

result = np.array([proj_b1, proj_b2, proj_b3])
result

array([3.74165739, 2.23606798, 8.36660027])

In [21]:
(b1 / np.linalg.norm(b1))

array([14, 28, 42])

In [24]:
b1

array([0.26726124, 0.53452248, 0.80178373])